# Regressiomalli

Sovitetaan myynnnin ennustava malli.

JAMK examples #8: 08_ml_predict_sales

Sisältää mm.

- sarakkeiden sisältämien NA-arvojen laskeminen
- regressiomallin mittarit 

Ladataan käytettävät paketit.

In [36]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

Luetaan käytettävä aineisto:

In [2]:
csvUrl = "https://raw.githubusercontent.com/RaimoHaikari/paikkatieto/master/misc/sales_data.csv"
df = pd.read_csv(csvUrl)

df.shape

(1000, 6)

Kuinka paljon aineistosta löytyy NA-arvoja?

In [3]:
# df.count()
df.isna().sum()

date                 0
category           183
sales                0
price                0
customer_age         0
customer_gender    323
dtype: int64

category- ja customer_gender-sarakkeissa NA-arvoja näyttäisi olevan. Tiputetaan NA-arvoja sisältävät tietueet pois.

Lisäksi: 

- muunnetaan date-sarake aikaleimaksi. Kannattaa lisätä reset_index() perään, niin välttää virheilmoituksen.
- lasketaan päivittäinen liikevaihto
- muutetaan aineisto aikasarjaksi

In [21]:
# Handle missing values
# df = df.dropna()
df = df.dropna().reset_index()

# Convert the 'date' column to datetime
df['date'] = pd.to_datetime(df['date'])

# Calculate the daily revenue
df['revenue'] = df['sales'] * df['price']

df.set_index('date', inplace=True)

Erotetaan itsenäiset- ja selitettävä muuttuja omiksi muuttujikeen.

In [5]:
# Prepare the data
X = df[['price', 'category']]  # Features
y = df['sales']  # Target variable


### ColumnTransformer

**column_transformer**:ia käytetään aina, kun kehikossa on sarakkeita, jotka
tarvitsevat yksilöllistä esikäsittelyä.

Esim. nyt halutaan sarake **category** dummy-koodata, mutta **price** sarakkeelle ei tehdä mitään.

Muutetaan category sarakkeen arvo muuttujatyypeiksi.

In [13]:
# Perform one-hot encoding on the 'category' column
# ct = ColumnTransformer([('encoder', OneHotEncoder(), [1])], remainder='passthrough')
# X = ct.fit_transform(X)

column_trans = make_column_transformer(
    (OneHotEncoder(), ['category']),
    remainder='passthrough'
)

X = column_trans.fit_transform(X)

Jaetaan aineisto opetus- ja testijoukkoihin.

In [15]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### LinearRegression

Alustetaan ja sovitetaan [LinearRegression -malli](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html).

In [16]:
# Initialize the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

Testataan mallin ennustuskykyä testiaineistolla.

In [31]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Keskiarvon ennustava vertailumalli, jolloin R^2 0
y_pred_mean = np.full(len(y_test),y_test.mean())

### Mittarinarvot

Tyypillisimpiä Regressiomallien mittareita ovat RMSE ja MSE -arvot.

Em. arvot lasketaan residuaalien perusteella, eli todellisten ja ennustettujen arvojen erotuksen mukaan.

- **MSE** korotetaan residuaalit toiseen ja lasketaan näiden summa.
- **RMSE** ottaan neliöjuuri edellä lasketusta arvosta.

RMSE:n arvo tulkitaan yleensä siten, että se kertoo kuinka paljon residuaalit keskimäärin poikkesivat nollasta.

**R^2 -arvoa** puolestan tulkitaan kertovan kuinka suuren osan vastemuuttujan arvojen vaihtelusta mallin kykenee selittämään.

Yksinkertaisimmillaan R^2 saadaan laskemalla ennustettujen ja todellisten arvojen välinen korrelaatio - näin ollen on syytä muistaa, että kyseessä on korrelaation arvo, ei niinkään tarkkuuden.

In [40]:
r2_score(y_test, y_pred_mean)
# r2_score(y_test, y_pred)

0.0

In [33]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', round(mse, 3))

Mean Squared Error: 35.931


In [20]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', round(rmse, 3))

Root Mean Squared Error: 5.994


In [38]:
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {round(mae, 3)}")

Mean Absolute Error: 5.295
